# Debug Notebook for GitHub Copilot Usage

This notebook demonstrates how to instantiate and invoke methods from your local exporter scripts. You can use it to debug the data extraction and adaptation processes for GitHub Copilot usage data.

In [1]:
# If your project structure is something like:
# PROJECT_ROOT/
#   |-- notebooks/debug_copilot_usage.ipynb
#   |-- src/
#        |-- extractor/
#             |-- github_exporter.py
#             |-- github_copilot_exporter.py
#        |-- common/
#             |-- common.py
#        |-- ...
# make sure to add the project root to sys.path if needed.

import sys
import os
project_root = os.path.abspath(os.path.join('..'))  # Adjust path if needed
if project_root not in sys.path:
    sys.path.append(project_root)

# Now we can import our local modules:
from src.extractor.github_copilot_exporter import GithubCopilotExporter
import pandas as pd

pd.options.mode.chained_assignment = None

ModuleNotFoundError: No module named 'requests'

## Prepare a Configuration Dictionary
Below is a sample configuration dictionary. Adjust the values according to your project’s needs or how you manage configs (e.g., from a .ini or .env file).

In [ ]:
config = {
    "GITHUB": {
        "github_url": "https://api.github.com",
        "github_org": "gologic-partner",     # Replace with your organization
        "github_token": "",  # Replace with your token
    },
    # Other config sections if needed...
}

# Instantiate the Exporter
copilot_exporter = GithubCopilotExporter()
copilot_exporter.initialize_data(config)

## Call the Copilot Usage Endpoint
We’ll do a simple request to GitHub’s Copilot usage endpoint. Adjust the query parameters (e.g., since, until) as needed.

For reference, see: [Copilot Usage Docs](https://docs.github.com/en/rest/copilot/copilot-usage?apiVersion=2022-11-28).

In [ ]:
endpoint = f"orgs/{copilot_exporter.github_org}/copilot/metrics"
# If you want to pass query parameters, you could do something like:
# endpoint = f"orgs/{copilot_exporter.github_org}/copilot/usage?since=2023-01-01&until=2023-01-31"

usage_data = copilot_exporter.execute_simple_request(endpoint)

print("Raw Usage Data (truncated):")
print(usage_data[:1])  # Display only the first element if it’s a list

## Adapt the Usage Data
Here we call the `adapt_usage` (which you can define in your exporter module following the same pattern as other `adapt_*` methods) to structure the data into a DataFrame or DataFrames.

In [ ]:
# In case you already have a method like 'adapt_usage' in your GithubCopilotExporter
# you can call it directly. Otherwise, define it similarly to how you have adapt_pulls, etc.

adapted_usage = copilot_exporter.adapt_usage(usage_data)

# The result might be a dict of DataFrames:
df_usage = adapted_usage["df_usage"]
df_usage_breakdown = adapted_usage["df_usage_breakdown"]

print("DataFrame: df_usage")
display(df_usage.head())

print("DataFrame: df_usage_breakdown")
display(df_usage_breakdown.head())

In [ ]:
# Cell 2 - Code
# First get the teams
teams = copilot_exporter.extract_teams()
print("\nTeams found:")
for team in teams:
    print(f"- {team['name']} (slug: {team['slug']})")

In [ ]:
# Cell 3 - Code
# Get metrics for each team
metrics_per_team = copilot_exporter.extract_metrics_per_team(teams)

print("\nMetrics per team:")
for team_slug, metrics in metrics_per_team.items():
    print(f"\nTeam: {team_slug}")
    print(f"Number of data points: {len(metrics)}")
    if len(metrics) > 0:
        print("Sample metrics:")
        print(metrics[0])

## Conclusion
You can now inspect these DataFrames, observe schema changes, and debug any issues with your transformation logic or with the GitHub API data itself. Use additional cells as needed to explore data, handle errors, etc.